In [1]:
import os 
os.environ["CUDA_VISIBLE_DEVICES"]="0";

In [2]:
import tensorflow as tf

tf.test.gpu_device_name()

'/device:GPU:0'

In [3]:
#######################################################
# Model Training
# 11/09/2019 
#
#######################################################
import numpy as np
import sklearn.metrics as metrics
import os
from math import floor


from tensorflow.keras.callbacks import ModelCheckpoint,CSVLogger
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from ISY5002_CA2_02_ModelDefinitions import createModel, my_preprocess

from tensorflow.keras.applications.vgg16 import preprocess_input as VGG16_preprocess_input
from tensorflow.keras.applications.resnet50 import preprocess_input as ResNet50_preprocess_input
from tensorflow.keras.applications.inception_v3 import preprocess_input as InceptionV3_preprocess_input

# import sys

# defining global variables
DEBUG_MODE = False
image_path = "./Resized"
seed        = 29 # fix random seed for reproducibility
np.random.seed(seed)
optmz       = 'adam'    # optimizers.RMSprop(lr=0.0001)
modelnameBase   = 'CA2'
num_classes = 3

channel = 3
num_epochs = 100

# hyperparameters
bsize = 32
rng_rot = 0
rng_zoom = 0.1
rng_bright = [0.01, 0]
rng_channel = 0.1
opt_hflip = True
opt_vflip = False

def create_summarise_plot(size, channel, index = 0):
    imgrows = size
    imgclms = size
    model = createModel(imgrows, imgclms, channel, index) # for training
    # modelGo = createModel() # for final testing
    model.summary()

    # Plot structure of network
    #from tensorflow.keras.utils import plot_model
    #plot_model(model, to_file='ISY5002_CA2_NN_' + str(index) + '.pdf', show_shapes=True, show_layer_names=False, rankdir='TB') 

    return model

def createIterators(size, index=0):
    imgrows = size
    imgclms = size

    preprocessing_fn = my_preprocess
    if (index > 90):
        # Using pretrained datasets - have to use their preprocess functions
        if (index == 90):
            preprocessing_fn = VGG16_preprocess_input
        elif (index == 91):
            preprocessing_fn = ResNet50_preprocess_input
        elif (index == 92):
            preprocessing_fn = InceptionV3_preprocess_input
    datagen = ImageDataGenerator(preprocessing_function = preprocessing_fn)
    datagenTrain = ImageDataGenerator(preprocessing_function = preprocessing_fn, 
        rotation_range=rng_rot, zoom_range=rng_zoom, 
        horizontal_flip=opt_hflip, vertical_flip=opt_vflip) 
        #brightness_range = rng_bright, channel_shift_range = rng_channel)
    
    train_it = datagenTrain.flow_from_directory('./Resized/train/', class_mode='categorical', target_size=(imgrows, imgclms), batch_size=bsize, color_mode='rgb')
    val_it = datagen.flow_from_directory('./Resized/validation/', class_mode='categorical', target_size=(imgrows, imgclms), batch_size=bsize, color_mode='rgb')
    # test_it = datagen.flow_from_directory('./Resized/test/', class_mode='categorical', target_size=(imgrows, imgclms), batch_size=1, color_mode='rgb')

    n_train = sum([len(files) for r, d, files in os.walk('./Resized/train/')])
    n_val = sum([len(files) for r, d, files in os.walk('./Resized/validation/')])
    # n_test = sum([len(files) for r, d, files in os.walk('./Resized/test/')])

    batchX, batchy = train_it.next()
    print('Batch shape=%s, min=%.3f, max=%.3f' % (batchX.shape, batchX.min(), batchX.max()))

    return train_it, val_it, n_train, n_val  



def main():

    # ------ CHANGE THESE ------
    index = 16
    size = 224
    # --------------------------


    modelname = modelnameBase + "_" + str(index) + "_" + str(size)

    # Create model and summary
    train_it, val_it, n_train, n_val= createIterators(size, index)
    model = create_summarise_plot(size, channel, index)

    # Create checkpoint for the training
    # This checkpoint performs model saving when
    # an epoch gives highest testing accuracy
    filepath        = modelname + ".hdf5"
    checkpoint      = ModelCheckpoint(filepath, monitor='val_acc', verbose=0, save_best_only=True, mode='max')

    # Log the epoch detail into csv
    csv_logger      = CSVLogger(modelname +'.csv')
    callbacks_list  = [checkpoint,csv_logger]
    
    # steps_per_epoch = total training data across all classes / batch size
    # validation_steps = number of batches in validation dataset defining 1 epoch
    model.fit_generator(
        train_it, steps_per_epoch=floor(n_train/bsize), 
        validation_data=val_it, 
        validation_steps=floor(n_val/bsize),
        epochs=num_epochs, callbacks = callbacks_list )

main()


Found 2496 images belonging to 3 classes.
Found 624 images belonging to 3 classes.
Batch shape=(32, 224, 224, 3), min=-1.841, max=2.330
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Inpt_conv (Conv2D)              (None, 224, 224, 16) 448         input_1[0][0]                    
__________________________________________________________________________________________________
Inpt_bn (BatchNormalization)    (None, 224, 224, 16) 64          Inpt_conv[0][0]                  
_____________________________________

Epoch 1/100
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
78/78 [==============================] - 44s 563ms/step - loss: 12.2859 - acc: 0.6143 - val_loss: 9.2894 - val_acc: 0.5658
Epoch 2/100
78/78 [==============================] - 28s 361ms/step - loss: 6.7706 - acc: 0.6214 - val_loss: 5.5320 - val_acc: 0.6420
Epoch 3/100
78/78 [==============================] - 28s 362ms/step - loss: 4.5767 - acc: 0.6542 - val_loss: 3.8711 - val_acc: 0.6727
Epoch 4/100
78/78 [==============================] - 28s 365ms/step - loss: 3.3652 - acc: 0.6794 - val_loss: 2.9303 - val_acc: 0.6963
Epoch 5/100
78/78 [==============================] - 28s 364ms/step - loss: 2.5882 - acc: 0.6911 - val_loss: 2.2601 - val_acc: 0.7133
Epoch 6/100
78/78 [==============================] - 28s 361ms/step - loss: 2.0293 - acc: 0.7141 - val_loss: 1.8337 - val_acc: 0.6974
Epoch 7/100
78/78 [==============================] - 28s 364ms/step - loss: 1.6571 - acc: 0.7292 - va

78/78 [==============================] - 28s 364ms/step - loss: 0.5422 - acc: 0.8604 - val_loss: 1.4534 - val_acc: 0.6820
Epoch 60/100
78/78 [==============================] - 28s 364ms/step - loss: 0.5295 - acc: 0.8599 - val_loss: 0.6713 - val_acc: 0.7654
Epoch 61/100
78/78 [==============================] - 28s 365ms/step - loss: 0.5211 - acc: 0.8610 - val_loss: 0.6705 - val_acc: 0.8021
Epoch 62/100
78/78 [==============================] - 29s 368ms/step - loss: 0.5128 - acc: 0.8612 - val_loss: 0.5583 - val_acc: 0.8438
Epoch 63/100
78/78 [==============================] - 28s 365ms/step - loss: 0.5235 - acc: 0.8558 - val_loss: 0.6126 - val_acc: 0.8394
Epoch 64/100
78/78 [==============================] - 28s 364ms/step - loss: 0.5137 - acc: 0.8646 - val_loss: 0.5737 - val_acc: 0.8322
Epoch 65/100
78/78 [==============================] - 29s 369ms/step - loss: 0.5264 - acc: 0.8563 - val_loss: 0.5824 - val_acc: 0.8503
Epoch 66/100
78/78 [==============================] - 29s 365ms/step